In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import config 
import sql_con
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import csv

In [2]:
# get module variables
ROOT_DIR = config.ROOT_DIR
select_records = sql_con.select_records
insert_records = sql_con.insert_records
update_records = sql_con.update_records
conn_odbc = sql_con.conn_odbc
read_contents = sql_con.read_contents

In [3]:
# function to make calls to cryptocompare API
def get_data(url, parameters, headers):
    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [4]:
# function to get api data for top 10 coins
def get_coin_data(coin_list, url, headers):
    coin_data = []
    parameters = { 
        "tsym":"USD",
        "allData":"true"
    }
    for coin in coin_list:
        parameters["fsym"] = coin
        res_json = get_data(url, parameters, headers)
        data = res_json["Data"]
        # iterate through the data and add the coin name to each row
        for row in data:
            row["symbol"] = coin
        coin_data.extend(data)
    return coin_data

In [5]:
# authorization header for making calls to crypto compare API
# read api key from config file using dotev module
headers = {
  "authorization": f"Apikey {config.API_KEY}"
}

In [6]:
# relevant urls for making calls to crypto compare API
top10_url = "https://min-api.cryptocompare.com/data/top/mktcapfull"
hist_url = "https://min-api.cryptocompare.com/data/histoday"

In [7]:
# get top 10 coins by market cap, capture json response
parameters = {
  "tsym":"USD",
  "limit": 10
}

res_json_top10 = get_data(top10_url, parameters, headers)
data_top10 = res_json_top10["Data"]

In [8]:
# capture top 10 coins in a list of dictionaries and write to json file (ingestion layer)
top10_coins = [{"Name": coin["CoinInfo"]["Name"], "FullName": coin["CoinInfo"]["FullName"], "Algorithm": coin["CoinInfo"]["Algorithm"], "ProofType": coin["CoinInfo"]["ProofType"]} for coin in data_top10]
with open(rf"{ROOT_DIR}/data/top10_coins.json", "w") as f:
    f.write(json.dumps(top10_coins))

In [9]:
# make request to cryptocompare api to get historical data for bitcoin quote prices in USD

# parameters = {
#   "fsym": "BTC",
#   "tsym":"USD",
#   "allData":"true"
# }

# res_json = get_data(hist_url, parameters, headers)
# data = res_json["Data"]

In [10]:
import findspark
findspark.init()

In [11]:
# spark session start to begin transforming data (processing layer)
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("crypto_analysis").getOrCreate()

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

schema = StructType([
    StructField("Name", StringType(), True),
    StructField("FullName", StringType(), True),
    StructField("Algorithm", StringType(), True),
    StructField("ProofType", StringType(), True)
])
df_top10 = spark.read.option("schema", schema).json(rf"{ROOT_DIR}\data\top10_coins.json").select("Name", "FullName", "Algorithm", "ProofType")
df_top10.show(truncate=False)

+----+------------+---------+---------+
|Name|FullName    |Algorithm|ProofType|
+----+------------+---------+---------+
|BTC |Bitcoin     |SHA-256  |PoW      |
|ETH |Ethereum    |Ethash   |PoS      |
|USDT|Tether      |N/A      |N/A      |
|XRP |XRP         |N/A      |XRP LCP  |
|BNB |Binance Coin|BEP-2    |PoSA     |
|USDC|USD Coin    |N/A      |N/A      |
|ADA |Cardano     |Ouroboros|PoS      |
|DOGE|Dogecoin    |Scrypt   |PoW      |
|APT |Aptos       |N/A      |N/A      |
|SOL |Solana      |N/A      |PoH      |
+----+------------+---------+---------+



In [13]:
coin_list = df_top10.rdd.map(lambda x: x[0]).collect()
coin_list

['BTC', 'ETH', 'USDT', 'XRP', 'BNB', 'USDC', 'ADA', 'DOGE', 'APT', 'SOL']

In [14]:
# read historical data for top 10 coins from cryptocompare API
all_coins_data = get_coin_data(coin_list, hist_url, headers)

In [119]:
# capture data from reponse and write to json file (ingestion layer)
with open(rf"{ROOT_DIR}\data\all_coins_data.json", "w") as f:
    f.write(json.dumps(all_coins_data))

In [120]:
# read ingested json file and print out first 10 records

schema = StructType([
    StructField("symbol", StringType(), True),
    StructField("time", LongType(), True),
    StructField("close", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("open", DoubleType(), True),
    StructField("volumefrom", DoubleType(), True),
    StructField("volumeto", DoubleType(), True),
    StructField("conversionType", StringType(), True),
    StructField("conversionSymbol", StringType(), True)
])

df = spark.read.option("schema", schema).json(rf"{ROOT_DIR}/data/all_coins_data.json")
df.show(n=10)

+-------+----------------+--------------+-------+-------+-------+------+----------+----------+--------+
|  close|conversionSymbol|conversionType|   high|    low|   open|symbol|      time|volumefrom|volumeto|
+-------+----------------+--------------+-------+-------+-------+------+----------+----------+--------+
|0.04951|                |        direct|0.04951|0.04951|0.04951|   BTC|1279324800|      20.0|  0.9902|
|0.08584|                |        direct|0.08585|0.05941|0.04951|   BTC|1279411200|     75.01|   5.092|
| 0.0808|                |        direct|0.09307|0.07723|0.08584|   BTC|1279497600|     574.0|   49.66|
|0.07474|                |        direct|0.08181|0.07426| 0.0808|   BTC|1279584000|     262.0|   20.59|
|0.07921|                |        direct|0.07921|0.06634|0.07474|   BTC|1279670400|     575.0|   42.26|
| 0.0505|                |        direct|0.08181| 0.0505|0.07921|   BTC|1279756800|    2160.0|  129.78|
|0.06262|                |        direct|0.06767| 0.0505| 0.0505

In [121]:
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- conversionSymbol: string (nullable = true)
 |-- conversionType: string (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- symbol: string (nullable = true)
 |-- time: long (nullable = true)
 |-- volumefrom: double (nullable = true)
 |-- volumeto: double (nullable = true)



In [122]:
# get ingestion date as current unix epoch time
# write data to csv file after adding ingestion date (csv ingestion point for data pipeline)
from pyspark.sql.functions import unix_timestamp, from_unixtime, col

df = df.withColumn("ingestion_date (unix epoch)", unix_timestamp()).withColumnRenamed("time", "time (unix epoch)")
df.write.mode("overwrite").option("quote", "\u0000").option("emptyValue", "").csv(rf"{ROOT_DIR}/data/all_coins_data.csv", header=True)

In [123]:
df.show(n=10, truncate=False)

+-------+----------------+--------------+-------+-------+-------+------+-----------------+----------+--------+---------------------------+
|close  |conversionSymbol|conversionType|high   |low    |open   |symbol|time (unix epoch)|volumefrom|volumeto|ingestion_date (unix epoch)|
+-------+----------------+--------------+-------+-------+-------+------+-----------------+----------+--------+---------------------------+
|0.04951|                |direct        |0.04951|0.04951|0.04951|BTC   |1279324800       |20.0      |0.9902  |1680564384                 |
|0.08584|                |direct        |0.08585|0.05941|0.04951|BTC   |1279411200       |75.01     |5.092   |1680564384                 |
|0.0808 |                |direct        |0.09307|0.07723|0.08584|BTC   |1279497600       |574.0     |49.66   |1680564384                 |
|0.07474|                |direct        |0.08181|0.07426|0.0808 |BTC   |1279584000       |262.0     |20.59   |1680564384                 |
|0.07921|                |d

In [124]:
# read from ingested csv file and print out first 10 records
schema = StructType([
    StructField("close", DoubleType(), True),
    StructField("conversionSymbol", StringType(), True),
    StructField("conversionType", StringType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("open", DoubleType(), True),
    StructField("symbol", StringType(), True),
    StructField("time (unix epoch)", LongType(), True),
    StructField("volumefrom", DoubleType(), True),
    StructField("volumeto", DoubleType(), True),
    StructField("ingestion_date (unix epoch)", LongType(), True)
])

df = spark.read.format("csv").option("header", "true").option("delimiter", ",").schema(schema).load(rf"{ROOT_DIR}/data/all_coins_data.csv")
#df = spark.read.csv(rf"{ROOT_DIR}/data/all_coins_data.csv", header=True)
df.show(n=10,truncate=False)

+-------+----------------+--------------+-------+-------+-------+------+-----------------+----------+--------+---------------------------+
|close  |conversionSymbol|conversionType|high   |low    |open   |symbol|time (unix epoch)|volumefrom|volumeto|ingestion_date (unix epoch)|
+-------+----------------+--------------+-------+-------+-------+------+-----------------+----------+--------+---------------------------+
|0.04951|null            |direct        |0.04951|0.04951|0.04951|BTC   |1279324800       |20.0      |0.9902  |1680564383                 |
|0.08584|null            |direct        |0.08585|0.05941|0.04951|BTC   |1279411200       |75.01     |5.092   |1680564383                 |
|0.0808 |null            |direct        |0.09307|0.07723|0.08584|BTC   |1279497600       |574.0     |49.66   |1680564383                 |
|0.07474|null            |direct        |0.08181|0.07426|0.0808 |BTC   |1279584000       |262.0     |20.59   |1680564383                 |
|0.07921|null            |d

In [125]:
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- conversionSymbol: string (nullable = true)
 |-- conversionType: string (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- symbol: string (nullable = true)
 |-- time (unix epoch): long (nullable = true)
 |-- volumefrom: double (nullable = true)
 |-- volumeto: double (nullable = true)
 |-- ingestion_date (unix epoch): long (nullable = true)



In [126]:
# extract necessary columns
df = df.select(["symbol", "time (unix epoch)", "open", "close", "high", "low", "volumefrom", "volumeto"])
df.dtypes

[('symbol', 'string'),
 ('time (unix epoch)', 'bigint'),
 ('open', 'double'),
 ('close', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('volumefrom', 'double'),
 ('volumeto', 'double')]

In [127]:
# get descriptive statistics for numeric columns
df.describe(["symbol", "open", "close", "high", "low", "volumefrom", "volumeto"]).show()

+-------+------+-----------------+-----------------+-----------------+-----------------+--------------------+-------------------+
|summary|symbol|             open|            close|             high|              low|          volumefrom|           volumeto|
+-------+------+-----------------+-----------------+-----------------+-----------------+--------------------+-------------------+
|  count| 46440|            46440|            46440|            46440|            46440|               46440|              46440|
|   mean|  null|946.4146105644052|947.0567881728905|972.5415717613753|917.4431752445142|2.4762585907576047E7|9.247115327000473E7|
| stddev|  null| 5303.62724937596|5305.099174289624|5446.683901175829|5141.800112725823| 1.596584600208285E8|3.553476900588219E8|
|    min|   ADA|              0.0|              0.0|              0.0|              0.0|                 0.0|                0.0|
|    max|   XRP|         67549.14|         67549.14|         68978.64|         66312.42|  

In [128]:
df.show(n=10)

+------+-----------------+-------+-------+-------+-------+----------+--------+
|symbol|time (unix epoch)|   open|  close|   high|    low|volumefrom|volumeto|
+------+-----------------+-------+-------+-------+-------+----------+--------+
|   BTC|       1279324800|0.04951|0.04951|0.04951|0.04951|      20.0|  0.9902|
|   BTC|       1279411200|0.04951|0.08584|0.08585|0.05941|     75.01|   5.092|
|   BTC|       1279497600|0.08584| 0.0808|0.09307|0.07723|     574.0|   49.66|
|   BTC|       1279584000| 0.0808|0.07474|0.08181|0.07426|     262.0|   20.59|
|   BTC|       1279670400|0.07474|0.07921|0.07921|0.06634|     575.0|   42.26|
|   BTC|       1279756800|0.07921| 0.0505|0.08181| 0.0505|    2160.0|  129.78|
|   BTC|       1279843200| 0.0505|0.06262|0.06767| 0.0505|    2402.5|  141.07|
|   BTC|       1279929600|0.06262|0.05454|0.06161|0.05049|    496.32|   26.73|
|   BTC|       1280016000|0.05454| 0.0505|0.05941| 0.0505|   1551.48|   85.06|
|   BTC|       1280102400| 0.0505|  0.056|  0.056|  

In [129]:
# set spark session timezone to UTC to have a uniform reference point for all date related fields
spark.conf.set("spark.sql.session.timeZone", "UTC")
df = df.withColumn("date_time (unix)", from_unixtime("time (unix epoch)", "yyyy-MM-dd HH:mm:ss"))
spark.conf.unset("spark.sql.session.timeZone")

In [130]:
# timezone will default to system timezone (Easter Standard Time) in absence of specific spark.sql.session.timeZone setting
df.withColumn("date_time", from_unixtime("time (unix epoch)", "yyyy-MM-dd HH:mm:ss")).show(n=10, truncate=False)

+------+-----------------+-------+-------+-------+-------+----------+--------+-------------------+-------------------+
|symbol|time (unix epoch)|open   |close  |high   |low    |volumefrom|volumeto|date_time (unix)   |date_time          |
+------+-----------------+-------+-------+-------+-------+----------+--------+-------------------+-------------------+
|BTC   |1279324800       |0.04951|0.04951|0.04951|0.04951|20.0      |0.9902  |2010-07-17 00:00:00|2010-07-16 20:00:00|
|BTC   |1279411200       |0.04951|0.08584|0.08585|0.05941|75.01     |5.092   |2010-07-18 00:00:00|2010-07-17 20:00:00|
|BTC   |1279497600       |0.08584|0.0808 |0.09307|0.07723|574.0     |49.66   |2010-07-19 00:00:00|2010-07-18 20:00:00|
|BTC   |1279584000       |0.0808 |0.07474|0.08181|0.07426|262.0     |20.59   |2010-07-20 00:00:00|2010-07-19 20:00:00|
|BTC   |1279670400       |0.07474|0.07921|0.07921|0.06634|575.0     |42.26   |2010-07-21 00:00:00|2010-07-20 20:00:00|
|BTC   |1279756800       |0.07921|0.0505 |0.0818

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

In [131]:
df2 = df.withColumn("HV Ratio", col("high")/col("volumefrom"))

# display dataframe in descending order of HV Ratio
df2.sort("HV Ratio", ascending=False).show(truncate=False)

+------+-----------------+------+------+------+------+----------+--------+-------------------+------------------+
|symbol|time (unix epoch)|open  |close |high  |low   |volumefrom|volumeto|date_time (unix)   |HV Ratio          |
+------+-----------------+------+------+------+------+----------+--------+-------------------+------------------+
|USDT  |1424476800       |1.15  |1.5   |1.5   |1.15  |1.415E-4  |2.122E-4|2015-02-21 00:00:00|10600.706713780919|
|USDT  |1441152000       |0.96  |1.15  |1.15  |0.96  |1.252E-4  |1.44E-4 |2015-09-02 00:00:00|9185.303514376996 |
|USDT  |1456185600       |1.1   |1.1   |1.1   |1.1   |1.518E-4  |1.67E-4 |2016-02-23 00:00:00|7246.376811594203 |
|USDT  |1420761600       |1.0   |1.0   |1.0   |1.0   |2.4E-4    |2.4E-4  |2015-01-09 00:00:00|4166.666666666667 |
|USDT  |1422662400       |0.9274|4.37  |4.37  |0.874 |0.005     |0.02185 |2015-01-31 00:00:00|874.0             |
|USDT  |1441065600       |0.96  |0.96  |0.96  |0.96  |0.002304  |2.301   |2015-09-01 00:

In [132]:
# sort by date_time (unix) in descending order to get HV Ratiio for most recent dates
df2.sort("date_time (unix)", ascending=False).show(truncate=False)

+------+-----------------+--------+--------+--------+--------+---------------+--------------+-------------------+----------------------+
|symbol|time (unix epoch)|open    |close   |high    |low     |volumefrom     |volumeto      |date_time (unix)   |HV Ratio              |
+------+-----------------+--------+--------+--------+--------+---------------+--------------+-------------------+----------------------+
|BTC   |1680480000       |28186.76|27899.45|28494.64|27290.26|50055.57       |1.4019177024E9|2023-04-03 00:00:00|0.5692601242978553    |
|XRP   |1680480000       |0.5185  |0.4999  |0.5242  |0.4844  |8.015672379E7  |4.069469086E7 |2023-04-03 00:00:00|6.539688440527267E-9  |
|BNB   |1680480000       |313.85  |307.39  |314.19  |300.12  |12143.88       |3762979.66    |2023-04-03 00:00:00|0.025872291228174193  |
|USDT  |1680480000       |1.0     |1.0     |1.0     |0.9998  |3.2038421362E8 |3.2043103595E8|2023-04-03 00:00:00|3.1212524134727684E-9 |
|ADA   |1680480000       |0.3817  |0.389 

In [133]:
df2.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- time (unix epoch): long (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volumefrom: double (nullable = true)
 |-- volumeto: double (nullable = true)
 |-- date_time (unix): string (nullable = true)
 |-- HV Ratio: double (nullable = true)



In [134]:
df2.groupby(col("symbol").alias("sym")).max("high").show()

+----+---------+
| sym|max(high)|
+----+---------+
| ETH|  4865.94|
|DOGE|   0.7398|
|USDC|    1.643|
| BTC| 68978.64|
|USDT|   207.09|
| XRP|     3.29|
| SOL|   259.95|
| BNB|   693.73|
| APT|    20.33|
| ADA|    3.097|
+----+---------+



#### What day had the Peak High in Price for each cryptocurrency in dataset?

In [135]:
df_max_high = df2.groupby(col("symbol").alias("sym")).agg({"high": "max"})
df2.join(df_max_high, (df2["symbol"] == df_max_high["sym"]) & (df2["high"] == df_max_high["max(high)"])).select(["symbol", "date_time (unix)", "high"]).show()

+------+-------------------+--------+
|symbol|   date_time (unix)|    high|
+------+-------------------+--------+
|   BTC|2021-11-10 00:00:00|68978.64|
|   ETH|2021-11-10 00:00:00| 4865.94|
|  USDT|2015-11-06 00:00:00|  207.09|
|   XRP|2018-01-04 00:00:00|    3.29|
|   BNB|2021-11-08 00:00:00|  693.73|
|  USDC|2020-03-12 00:00:00|   1.643|
|   ADA|2021-09-02 00:00:00|   3.097|
|  DOGE|2021-05-08 00:00:00|  0.7398|
|   APT|2023-01-30 00:00:00|   20.33|
|   SOL|2021-11-06 00:00:00|  259.95|
+------+-------------------+--------+



#### What is the mean closing price for each cryptocurrency?

In [139]:
df2.groupby("symbol").mean("close").sort("avg(close)", ascending=False).show()

+------+--------------------+
|symbol|          avg(close)|
+------+--------------------+
|   BTC|   8890.763590996981|
|   ETH|   501.0292704349697|
|   BNB|   65.51261989664081|
|   SOL|  11.368294465977598|
|  USDT|  0.7592086563307624|
|  USDC| 0.35299541343669266|
|   APT|  0.3212228682170544|
|   XRP| 0.22891996317829463|
|   ADA|  0.2054149397071489|
|  DOGE|0.026344093449612395|
+------+--------------------+

